## SARSA example of Cliff Walking

In [1]:
USE {
    repositories {
        mavenCentral()
        maven("https://central.sonatype.com/repository/maven-snapshots/")
    }
    dependencies {
        implementation("io.github.kotlinrl:integration:0.1.0-SNAPSHOT")
        implementation("io.github.kotlinrl:tabular:0.1.0-SNAPSHOT")
        implementation("io.github.kotlinrl:envs:0.1.0-SNAPSHOT")
        implementation("io.github.kotlinrl:rendering:0.1.0-SNAPSHOT")
    }
}

In [2]:
import io.github.kotlinrl.core.*
import io.github.kotlinrl.core.RecordVideo
import io.github.kotlinrl.core.api.*
import io.github.kotlinrl.core.wrapper.*
import io.github.kotlinrl.integration.gymnasium.*
import io.github.kotlinrl.integration.gymnasium.GymnasiumEnvs.*
import io.github.kotlinrl.rendering.*
import io.github.kotlinrl.tabular.*
import io.github.kotlinrl.tabular.td.classic.*
import org.jetbrains.kotlinx.kandy.letsplot.export.*
import org.jetbrains.kotlinx.multik.api.*
import org.jetbrains.kotlinx.multik.api.io.*
import org.jetbrains.kotlinx.multik.ndarray.data.*
import java.io.*


In [3]:
val maxStepsPerEpisode = 9_000
val trainingEpisodes = 500
val testEpisodes = 50
val initialEpsilon = 1.0
val epsilonDecayRate = 0.98
val minEpsilon = 0.1
val alpha = 0.5
val gamma = 0.99
val fileName = "CliffWalkingSARSA.npy"
val actionSymbols = mapOf(
    3 to "←",
    2 to "↓",
    1 to "→",
    0 to "↑"
)


In [4]:
val env = gymnasium.make<CliffWalkingEnv>(CliffWalking_v0, render = true, options = mapOf(
    "is_slippery" to false
))

var trainingQtable: QTable = mk.d2array(48, 4) { 0.0 }

val (epsilonSchedule, epsilonDecrement) = ParameterSchedule.geometricDecay(
    initialValue = initialEpsilon,
    minValue = minEpsilon,
    decayRate = epsilonDecayRate
)


2025-09-17T18:29:31.737606Z Execution of code 'val env = gymnasium....' ERROR Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


In [5]:
val trainer = episodicTrainer(
    env = env,
    agent = learningAgent(
        id = "training",
        algorithm = SARSA(
            Q = trainingQtable,
            epsilon = epsilonSchedule,
            alpha = ParameterSchedule.constant(alpha),
            gamma = gamma,
        )
    ),
    maxStepsPerEpisode = maxStepsPerEpisode,
    successfulTermination = { it.done },
    callbacks = listOf(
        printEpisodeStart(100),
        onEpisodeEnd { epsilonDecrement() }
    )
)
println("Starting training")
val training = trainer.train(maxEpisodes(trainingEpisodes))
mk.writeNPY(fileName, trainingQtable)


Starting training
Episode 1 failed to terminate and was cut short. maxStepsPerEpisode=9000  truncated=false
Starting episode 100
Starting episode 200
Starting episode 300
Starting episode 400
Starting episode 500
Max episodes reached: 500


In [6]:
val testingQtable = mk.readNPY<Double, D2>(fileName).asD2Array()


In [7]:
val recordEnv = RecordVideo(env = env, folder = "videos/cliff_walking_sarsa", testEpisodes / 3)
val tester = episodicTrainer(
    env = recordEnv,
    agent = policyAgent(
        id = "testing",
        policy = testingQtable.greedy()
    ),
    maxStepsPerEpisode = maxStepsPerEpisode,
    successfulTermination = { it.done },
    callbacks = listOf(
        printEpisodeStart(10)
    )
)
println("Starting testing")
val test = tester.train(maxEpisodes(testEpisodes))


Starting testing
Starting episode 10
Starting episode 20
Starting episode 30
Starting episode 40
Starting episode 50
Max episodes reached: 50


In [8]:
println("Training average reward: ${training.totalAverageReward}")
println("Test average reward: ${test.totalAverageReward}")

printQTable(testingQtable, 4, 12, actionSymbols = actionSymbols)
displayVideos(recordEnv.folder)


Training average reward: -621.182
Test average reward: -17.0
Action Value Function:
-80.78 -98.80 -34.55 -34.52 -26.17 -23.19  -9.28  -8.63  -7.00  -6.25  -4.49  -6.64 
-46.98 -39.03 -29.74 -96.69 -58.65 -215.58 -25.80 -23.12  -8.95 -13.96  -2.54  -1.09 
-57.24 -84.86 -85.20 -86.04 -125.76 -58.30 -347.67 -59.05 -121.04 -29.78  -7.66   0.00 
-71.89   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00 
Policy Table:
  ↓   ←   →   →   →   →   →   →   →   →   ↓   → 
  →   →   ↑   ↑   ↑   →   ↑   ↑   ↑   ↑   →   ↓ 
  ↑   ↑   ↑   ↑   ↑   ↑   ←   ↑   ↑   ↑   ↑   ↓ 
  ↑   ↑   ↑   ↑   ↑   ↑   ↑   ↑   ↑   ↑   ↑   ↑ 


In [9]:
plotPolicyActionValueGrid(testingQtable, 4, 12, actionSymbols )

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="fbGOaH"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"layout":{
"name":"grid",
"ncol":1,
"nrow":2,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"Action Value Function"
},
"mapping":{
},
"data":{
"x":[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0],
"y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0],
"label":["-80.78","-98.8","-34.55","-34.52","-26.17","-23.19","-9.28","-8.63","-7.0","-6.25","-4.49","-6.64","-46.98","-39.03","-29.74","-96.69","-58.65","-215.58","-25.8","-23.12","-8.95","-13.96","-2.54","-1.09","-57.24","-84.86","-85.2","-86.04","-125.76","-58.3","-347.67","-59.05","-121.04","-29.78","-7.66","0.0","-71.89","0.0","0.0","0.0","0.0","0.0","0.0","0.0","0.0","0.0","0.0","0.0"],
"value":[-80.78,-98.8,-34.55,-34.52,-26.17,-23.19,-9.28,-8.63,-7.0,-6.25,-4.49,-6.64,-46.98,-39.03,-29.74,-96.69,-58.65,-215.58,-25.8,-23.12,-8.95,-13.96,-2.54,-1.09,-57.24,-84.86,-85.2,-86.04,-125.76,-58.3,-347.67,-59.05,-121.04,-29.78,-7.66,0.0,-71.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_gradient",
"high":"#ffffff",
"low":"#5470c6",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"fill":"value"
},
"stat":"identity",
"size":0.5,
"color":"#000000",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"tile",
"data":{
}
},{
"mapping":{
"x":"x",
"y":"y",
"label":"label"
},
"stat":"identity",
"size":6.0,
"color":"#000000",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"text",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"int",
"column":"x"
},{
"type":"int",
"column":"y"
},{
"type":"float",
"column":"value"
},{
"type":"str",
"column":"label"
}]
},
"spec_id":"3"
},{
"ggtitle":{
"text":"Policy"
},
"mapping":{
},
"data":{
"x":[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0],
"y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0],
"label":["↓","←","→","→","→","→","→","→","→","→","↓","→","→","→","↑","↑","↑","→","↑","↑","↑","↑","→","↓","↑","↑","↑","↑","↑","↑","←","↑","↑","↑","↑","↓","↑","↑","↑","↑","↑","↑","↑","↑","↑","↑","↑","↑"],
"value":[-80.78,-98.8,-34.55,-34.52,-26.17,-23.19,-9.28,-8.63,-7.0,-6.25,-4.49,-6.64,-46.98,-39.03,-29.74,-96.69,-58.65,-215.58,-25.8,-23.12,-8.95,-13.96,-2.54,-1.09,-57.24,-84.86,-85.2,-86.04,-125.76,-58.3,-347.67,-59.05,-121.04,-29.78,-7.66,0.0,-71.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_gradient",
"high":"#ffffff",
"low":"#5470c6",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aest